In [1]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, 'plots')

import os

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets


import json
from SplunkIntelOptimized import SplunkIntelOptimized
from sources.SplunkDatasetNew import SplunkDatasetNew
from sources.SplunkFileSource import SplunkFileSource

import plotly as py
import pandas as pd
import numpy as np
import plotNew as plot

py.offline.init_notebook_mode()
pd.set_option('display.notebook_repr_html', True)
    
data_sources_names = []

data_source_picker = None
time_range_picker = None
event_picker = None
event_text_area = None 
threshold_picker = None
new_time_range_picker = None

#create widgets
def create_widgets():
    global data_source_picker, time_range_picker, event_picker
    global event_text_area, threshold_picker, new_time_range_picker
    for file in os.listdir("../../data"):
            if file.endswith(".json"):
                data_sources_names.append(file)

    data_source_picker = widgets.Dropdown(
            options=data_sources_names,
            description='Data Source:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Control Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    new_time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Test Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    threshold_picker = widgets.FloatSlider(
                value=0.8,
                min=0,
                max=1,
                step=0.1,
                description='Threshold:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
                slider_color='white',
                color='black'
        )
    
    event_picker = widgets.Dropdown(
            options=[1,2,3],
            description='Event Number:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )
    
    l = widgets.Layout(height='40px', width='800px')
    event_text_area = widgets.Textarea(value='TA: height=40px', layout=l)

    
def initialize(change):    
    # Just to set the widgets
    all_events = []
    all_events.extend(SplunkFileSource.load_data('../../data/' + data_source_picker.value))    
    minutes = 0
    for dict in all_events:
        if dict.get('cluster_label') == '1':
            minutes = minutes + 1

    time_range_picker.value = [1,15]
    time_range_picker.min = 1
    time_range_picker.max = minutes
    
    #new_time_range_picker.value = [minutes -2, minutes]
    new_time_range_picker.value = [1, 15]
    new_time_range_picker.min = 1
    new_time_range_picker.max = minutes 
    
    change_event_handler(None)
    
    
splunkDataset = SplunkDatasetNew()
splunkIntelargs = []

def event_picker_change_handler(change):
    event_text_area.value = json.dumps(splunkDataset.get_all_events_for_notebook()[event_picker.value])

def split(input, length, size):
    input.replace('\n', ' ')
    input.replace('\tat', ' ')
    return '<br>'.join([input[start:start + size] for start in range(0, length, size)])

def get_tool_tips(all_events):
    tooltips = []
    for idx, event in enumerate(all_events):
        tooltips.append(
            split(event[0], min(100, len(event[0])), 100) + '<br> id = ' + str(idx) + '<br> cluster = ' + str(
            event[3]))
            
    return tooltips
    
# Handle data source change
def change_event_handler(change):
    global splunkDataset
    
    control_start = time_range_picker.value[0]
    test_start = new_time_range_picker.value[0]

    clear_output()

    prev_out_file = None
    while(control_start <= time_range_picker.value[1] or test_start < new_time_range_picker.value[1]):

        del splunkIntelargs[:]
        splunkDataset = SplunkDatasetNew() 

        print(control_start, control_start)
        print(test_start, test_start)
        splunkDataset.load_legacy_file('../../data/' + data_source_picker.value,
                                     [control_start, control_start],
                                     [test_start, test_start], prev_out_file)
    
        splunkIntelargs.append('--sim_threshold')
        splunkIntelargs.append(str(float(threshold_picker.value)))
    
        print(splunkIntelargs)

        splunkIntel = SplunkIntelOptimized(splunkDataset, SplunkIntelOptimized.parse(splunkIntelargs))
        splunkDataset = splunkIntel.run()
    
        file_object  = open("result.json", "w")
        file_object.write(splunkDataset.get_output_for_notebook_as_json)
        file_object.close()
        
        prev_out_file = './result.json'
        
        control_start = control_start + 1
        test_start = test_start + 1

    xy_matrix, tooltips, labels,sizes = splunkDataset.control_scatter_plot()
    new_sizes = []
    for size in sizes:
        new_sizes.append((2 + len(str(size)))**2)
        
    print(new_sizes)
    plot.scatter_plot_groups(xy_matrix, labels, tooltips, ['control', 'test', 'test-anomaly', 'test-unx-freq'], ['blue','green','red', 'orange'])
    
    #xy_matrix, tooltips, labels, sizes = splunkDataset.count_scatter_plot()
    
    #new_sizes = []
    #for size in sizes:
     #   new_sizes.append((3 + len(str(size)))**2)
        
    #print(new_sizes)    
    
    #plot.scatter_plot_groups(xy_matrix, labels, tooltips, ['test', 'test-anomaly'], ['green','red'], new_sizes)


    #for key,data in splunkDataset.get_anom_clusters().items():
     #   for host,anom in data.items():
     #       print(host, anom.get('text'))
    
    xy_matrix, tooltips, labels, clusters = splunkDataset.control_scatter_plot_4d()
    plot.scatter_plot_groups_4d(xy_matrix, labels, clusters, tooltips, ['blue','orange','red'])
            
    event_picker.options = [ x for x in range(len(splunkDataset.get_all_events_for_notebook()))] 
     

create_widgets()

#intialize
initialize(None)
    
#setup even handlers    
data_source_picker.observe(initialize, names="value")
time_range_picker.observe(change_event_handler, names="value")
new_time_range_picker.observe(change_event_handler, names="value")
event_picker.observe(event_picker_change_handler, names="value")
threshold_picker.observe(change_event_handler, names="value")



# show widgets
display(widgets.HBox([data_source_picker]))
display(widgets.HBox([time_range_picker, threshold_picker]))
display(widgets.HBox([new_time_range_picker]))
display(widgets.HBox([event_picker, event_text_area]))

2017-07-12 12:31:56,110 INFO loading file ../../data/mzs.json


(1, 1)
(1, 1)


2017-07-12 12:31:56,366 INFO Running using file source
2017-07-12 12:31:56,367 INFO Start vectorization....
2017-07-12 12:31:56,368 INFO setting min_df = 0.05 and max_df = 1.0


['--sim_threshold', '0.8']
('control count = ', 100)


2017-07-12 12:31:59,039 INFO Running kemans with k = 50
2017-07-12 12:31:59,692 INFO found k = 50
2017-07-12 12:31:59,693 INFO Running kemans with k = 25
2017-07-12 12:32:00,115 INFO Running kemans with k = 37
2017-07-12 12:32:00,605 INFO found k = 37
2017-07-12 12:32:00,606 INFO Running kemans with k = 31
2017-07-12 12:32:01,071 INFO found k = 31
2017-07-12 12:32:01,071 INFO Running kemans with k = 28
2017-07-12 12:32:01,457 INFO found k = 28
2017-07-12 12:32:01,458 INFO Running kemans with k = 26
2017-07-12 12:32:01,819 INFO found k = 26


...

['--sim_threshold', '0.8']
('control count = ', 161)


2017-07-12 12:37:08,887 INFO Running kemans with k = 81
2017-07-12 12:37:09,858 INFO found k = 81
2017-07-12 12:37:09,858 INFO Running kemans with k = 40
2017-07-12 12:37:10,447 INFO Running kemans with k = 60
2017-07-12 12:37:11,218 INFO Running kemans with k = 70
2017-07-12 12:37:12,097 INFO found k = 70
2017-07-12 12:37:12,098 INFO Running kemans with k = 65
2017-07-12 12:37:12,937 INFO found k = 65
2017-07-12 12:37:12,938 INFO Running kemans with k = 62
2017-07-12 12:37:13,732 INFO Running kemans with k = 63
2017-07-12 12:37:14,547 INFO found k = 63


('test count = ', 161)


2017-07-12 12:37:27,450 INFO Detect Count Anomalies....
2017-07-12 12:37:27,451 INFO Using ZeroDeviationClassifier for cluster 0
2017-07-12 12:37:27,452 INFO Using ZeroDeviationClassifier for cluster 1
2017-07-12 12:37:27,453 INFO Using ConnectedSetClassifierNew for cluster 2
2017-07-12 12:37:27,462 INFO Using ZeroDeviationClassifier for cluster 3
2017-07-12 12:37:27,463 INFO Using ZeroDeviationClassifier for cluster 4
2017-07-12 12:37:27,464 INFO Using ZeroDeviationClassifier for cluster 5
2017-07-12 12:37:27,465 INFO Using ConnectedSetClassifierNew for cluster 6
2017-07-12 12:37:27,478 INFO Using ConnectedSetClassifierNew for cluster 7
2017-07-12 12:37:27,596 INFO Using ZeroDeviationClassifier for cluster 8
2017-07-12 12:37:27,597 INFO Using ConnectedSetClassifierNew for cluster 9
2017-07-12 12:37:27,600 INFO Using ConnectedSetClassifierNew for cluster 10
2017-07-12 12:37:27,604 INFO Using ConnectedSetClassifierNew for cluster 11
2017-07-12 12:37:27,610 INFO Using ConnectedSetClassif

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 25, 25, 25, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 36, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49,